Declaração de bibliotecas

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os

Classe Microfone

In [2]:
class Microfone:
    def __init__(self, position, ganho=1.0, resposta_em_frequencia=None, directividade=1.0, ruido_externo=0.0, ruido_interno=0.0):
        self.position = np.array(position)  # Posição do microfone (x, y)
        self.ganho = ganho
        self.resposta_em_frequencia = resposta_em_frequencia
        self.directividade = directividade
        self.ruido_externo = ruido_externo
        self.ruido_interno = ruido_interno

    def captar_sinal(self, sinal, delay=0):
        sinal_atrasado = np.roll(sinal, int(delay * len(sinal)))
        sinal_captado = (
            self.ganho
            * self.directividade
            * (sinal_atrasado if self.resposta_em_frequencia is None else np.convolve(sinal_atrasado, self.resposta_em_frequencia, mode='same'))
            + np.random.normal(0, self.ruido_externo + self.ruido_interno, len(sinal))
        )
        return sinal_captado

Calculo do Delay and Sum (DAS)

In [3]:
def delay_and_sum(distance, captured_signals, microphones):
    summed_signal = np.zeros_like(captured_signals)
    y_source_DAS = int(distance)

    for mic_idx, microphone in enumerate(microphones):
        for other_mic_idx, other_microphone in enumerate(microphones):
            delay = np.sqrt((other_microphone.position[0] - microphone.position[0])**2 + (other_microphone.position[1] - y_source_DAS)**2) / speed_of_sound
            summed_signal[mic_idx] += np.roll(captured_signals[other_mic_idx], int(delay * len(captured_signals[other_mic_idx])))
        summed_signal[mic_idx] = summed_signal[mic_idx] / captured_signals.shape[0]


    return summed_signal

In [4]:
def salvar_array_csv(nome_arquivo, diretorio, array):
    if not os.path.exists(diretorio):
        os.makedirs(diretorio)

    caminho_arquivo_csv = os.path.join(diretorio, nome_arquivo)

    cabecalho = ';'.join([f'Microfone_{i+1}' for i in range(array.shape[0])])

    array = array.T

    np.savetxt(caminho_arquivo_csv, array, delimiter=';', header=cabecalho)

Função para gerar um case especifico de microfones, posição da fonte e diferentes freqeucnias de sinal aplicados

In [5]:
def gerar_dataset():
    num_microphones_list = [64]#[16, 32, 64, 128]
    num_signals_list = [1, 2, 3, 4]
    sample_rate = 10000  # Hz

    num_microphones = np.random.choice(num_microphones_list)
    num_signals = np.random.choice(num_signals_list)
    y_position = np.random.uniform(0, 10)

    source_positions = [(np.random.uniform(0, 6), y_position) for _ in range(num_signals)]
    signals = []

    for _ in range(num_signals):
        frequency = np.random.uniform(200, 15000)
        t = np.arange(0, 1, 1/sample_rate)
        signal = np.sin(2 * np.pi * frequency * t)
        signals.append(signal)

    return np.array(source_positions), np.array(signals), num_microphones, y_position

Apena um teste para validar a função anterior

In [6]:
posicoes, sinais, num_microfones, y_position = gerar_dataset()
print("Posições:")
print(posicoes)
print("\nSinais:")
print(sinais)
print("\nNúmero de microfones:")
print(num_microfones)
print("\nY position:")
print(y_position)

Posições:
[[5.12356922 5.77303535]
 [4.09164738 5.77303535]]

Sinais:
[[ 0.          0.13433255 -0.26623001 ...  0.68067636 -0.576097
   0.46107452]
 [ 0.          0.82406311  0.93366021 ...  0.16880146  0.90786356
   0.85980432]]

Número de microfones:
64

Y position:
5.77303535191866


In [7]:
len(posicoes)

2

In [8]:
sinais.shape

(2, 10000)

In [9]:
def create_ideal_microphones(num_microphones, x_size, y_size, microphone_line_length):
    # Inicialização dos microfones com espaçamento ao longo da linha
    start_position_mic = (x_size - microphone_line_length) / 2
    microphones = [
        Microfone(
            position=[start_position_mic + (microphone_line_length / (num_microphones - 1) * i), y_size],
            ganho=1.2,
            resposta_em_frequencia= np.ones(100),   #np.random.normal(1, 0.1, 100),  # Exemplo de resposta em frequência aleatória
            directividade= 1.0,     #np.random.uniform(0.5, 1.0),  # Exemplo de directividade aleatória
            ruido_externo= 0.0,     #np.random.uniform(0.005, 0.02),  # Exemplo de ruído externo aleatório
            ruido_interno= 0.0      #np.random.uniform(0.001, 0.01)  # Exemplo de ruído interno aleatório
        )
        for i in range(num_microphones)
    ]
    return microphones

In [10]:
def create_real_microphones(num_microphones, x_size, y_size, microphone_line_length):
    # Inicialização dos microfones com espaçamento ao longo da linha
    start_position_mic = (x_size - microphone_line_length) / 2
    microphones = [
        Microfone(
            position=[start_position_mic + (microphone_line_length / (num_microphones - 1) * i), y_size],
            ganho=1.2,
            resposta_em_frequencia= np.random.normal(1, 0.1, 100),  # Exemplo de resposta em frequência aleatória
            directividade= np.random.uniform(0.5, 1.0),  # Exemplo de directividade aleatória
            ruido_externo= np.random.uniform(0.005, 0.02),  # Exemplo de ruído externo aleatório
            ruido_interno= np.random.uniform(0.001, 0.01)  # Exemplo de ruído interno aleatório
        )
        for i in range(num_microphones)
    ]
    return microphones

In [11]:
num_datasets = 100

In [12]:
x_size = 6  # metros
y_size = 10  # metros
microphone_line_length = 2  # metros
speed_of_sound = 343  # metros por segundo (velocidade do som)

sample_rate = 10000  # Hz
t = np.arange(0, 1, 1/sample_rate)

Função completa para a criação do dataset

In [13]:
def datasets(num_datasets):
    for i in range(num_datasets):
        source_positions, signal, num_microphones, y_position = gerar_dataset()
        print("Posições:")
        print(source_positions)
        print("\nSinais:")
        print(signal)
        print("\nNúmero de microfones:")
        print(num_microphones)
        print("\nY position:")
        print(y_position)

        ideal_microphones = create_ideal_microphones(num_microphones, x_size, y_size, microphone_line_length)
        real_microphones = create_real_microphones(num_microphones, x_size, y_size, microphone_line_length)

        delays = np.zeros((num_microphones, len(source_positions)))

        for mic_idx, microphone in enumerate(real_microphones):
            for src_idx, src_coord in enumerate(source_positions):
                distance = np.sqrt((src_coord[0] - microphone.position[0])**2 + (src_coord[1] - microphone.position[1])**2)
                delays[mic_idx, src_idx] = distance / speed_of_sound
        
        captured_signals_ideal = np.zeros((num_microphones, len(t)))
        for mic_idx, (microphone, delay_values) in enumerate(zip(ideal_microphones, delays)):
            for src_idx, delay in enumerate(delay_values):
                captured_signals_ideal[mic_idx, :] += microphone.captar_sinal(signal[src_idx], delay)

        captured_signals_real = np.zeros((num_microphones, len(t)))
        for mic_idx, (microphone, delay_values) in enumerate(zip(real_microphones, delays)):
            for src_idx, delay in enumerate(delay_values):
                captured_signals_real[mic_idx, :] += microphone.captar_sinal(signal[src_idx], delay)

        salvar_array_csv(f'cru_nm{num_microphones}_nss{len(source_positions)}_ypos{(1000*y_position):.0f}_ideal.csv', './64mics/cru', captured_signals_ideal)
        salvar_array_csv(f'cru_nm{num_microphones}_nss{len(source_positions)}_ypos{(1000*y_position):.0f}_real.csv', './64mics/cru', captured_signals_real)

        for i in range (0, 11):
            print(f'i: {i}')
            summed_signal = delay_and_sum(i, captured_signals_real, real_microphones)
            if i < y_position:
                print(f'antes_nm{num_microphones}_nss{len(source_positions)}_ypos{(1000*y_position):.0f}_posDAS{i}_real.csv')
                salvar_array_csv(f'antes_nm{num_microphones}_nss{len(source_positions)}_ypos{(1000*y_position):.0f}_posDAS{i}_real.csv', './64mics/antes', summed_signal)
            else:
                print(f'depois_nm{num_microphones}_nss{len(source_positions)}_ypos{(1000*y_position):.0f}_posDAS{i}_real.csv')
                salvar_array_csv(f'depois_nm{num_microphones}_nss{len(source_positions)}_ypos{(1000*y_position):.0f}_posDAS{i}_real.csv', './64mics/depois', summed_signal)

In [13]:
datasets(num_datasets)

Posições:
[[3.80485355 8.82876648]
 [0.40305734 8.82876648]
 [4.43824451 8.82876648]]

Sinais:
[[ 0.          0.96302445 -0.51890453 ...  0.99232376 -0.3864404
  -0.78409885]
 [ 0.          0.95845914  0.54676237 ...  0.23880793  0.99884302
   0.33099182]
 [ 0.          0.9999908   0.00857841 ...  0.45127863 -0.89043925
  -0.45891726]]

Número de microfones:
64

Y position:
8.828766478451044
i: 0
antes_nm64_nss3_ypos8829_posDAS0_real.csv
i: 1
antes_nm64_nss3_ypos8829_posDAS1_real.csv
i: 2
antes_nm64_nss3_ypos8829_posDAS2_real.csv
i: 3
antes_nm64_nss3_ypos8829_posDAS3_real.csv
i: 4
antes_nm64_nss3_ypos8829_posDAS4_real.csv
i: 5
antes_nm64_nss3_ypos8829_posDAS5_real.csv
i: 6
antes_nm64_nss3_ypos8829_posDAS6_real.csv
i: 7
antes_nm64_nss3_ypos8829_posDAS7_real.csv
i: 8
antes_nm64_nss3_ypos8829_posDAS8_real.csv
i: 9
depois_nm64_nss3_ypos8829_posDAS9_real.csv
i: 10
depois_nm64_nss3_ypos8829_posDAS10_real.csv
Posições:
[[5.23049116 3.57990359]
 [2.53635374 3.57990359]
 [4.20761885 3.57990359]

In [21]:
import numpy as np
from sklearn.decomposition import PCA

array_original = np.random.rand(64, 10000)

pca = PCA(n_components=1)
array_reduzido = pca.fit_transform(array_original.T)

print("Shape do array reduzido:", array_reduzido.shape)


Shape do array reduzido: (10000, 1)


In [24]:
array_original.T.shape

(10000, 64)